# 模型训练

In [6]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, concatenate, Conv2DTranspose, BatchNormalization, Activation, Dropout
from tensorflow.keras.optimizers import Adadelta, Nadam ,Adam
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.utils import  plot_model ,Sequence
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import load_img,img_to_array
import tensorflow as tf
from tensorflow.python.keras.losses import binary_crossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Dropout, BatchNormalization, Activation, Conv2DTranspose
from scipy.ndimage import morphology as mp
from PIL import Image,UnidentifiedImageError
from sklearn.model_selection import train_test_split
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import shutil
import os
from glob import glob  # for getting list paths of image and labels
from random import choice,sample
from matplotlib import pyplot as plt
import cv2 # saving and loading images

print("环境准备完毕")

环境准备完毕


### 获取数据集 

In [7]:
train_img_dir = '../autodl-tmp/Endovis18/Train/images/' 
train_mask_dir = '../autodl-tmp/Endovis18/Train/pixeled_annotations_train/'

train_imgs = os.listdir(train_img_dir)
train_masks = os.listdir(train_mask_dir)

### 数据对齐 

In [8]:
if len(train_imgs) != len(train_masks):
    print("The number of images and masks are not equal.")

    # 使用文件名，因为列表直接包含文件名
    img_set = set(train_imgs)
    mask_set = set(train_masks)

    extra_imgs = img_set - mask_set
    extra_masks = mask_set - img_set

    print("Extra images:", extra_imgs)
    print("Extra masks:", extra_masks)

    # 从图像和掩码列表中移除多余的条目
    train_imgs = [img for img in train_imgs if img not in extra_imgs]
    train_masks = [mask for mask in train_masks if mask not in extra_masks]

    print("Updated number of images:", len(train_imgs))
    print("Updated number of masks:", len(train_masks))


print(len(train_imgs))
print(len(train_masks))

The number of images and masks are not equal.
Extra images: {'seq_1_frame025.png'}
Extra masks: {'seq_10_frame047.png', 'seq_10_frame044.png', 'seq_10_frame045.png', 'seq_10_frame046.png', 'seq_10_frame048.png', 'seq_10_frame049.png'}
Updated number of images: 2228
Updated number of masks: 2228
2228
2228


### 划分数据集 

In [9]:
val_img_dir =  train_img_dir
val_mask_dir = train_mask_dir


train_imgs,val_imgs,train_masks,val_masks =  train_test_split(train_imgs, train_masks, test_size=0.13, random_state=42)


print(len(train_masks))
print(len(val_masks))

1938
290


### 数据集加载

In [10]:
num_batch_size = 8
class DataGenerator(Sequence):
    'Generates data for Keras'
    
    def __init__(self, images,image_dir,labels,label_dir ,batch_size, dim=(224,224,3) ,shuffle=True):
        'Initialization'
        self.dim = dim
        self.images = images
        self.image_dir = image_dir
        self.labels = labels
        self.label_dir = label_dir
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.images) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [k for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.images))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        batch_imgs = list()
        batch_labels = list()

        # Generate data
        for i in list_IDs_temp:
#             degree=np.random.random() * 360
            # Store sample
            img = load_img(self.image_dir + self.images[i] ,target_size=self.dim)
            img = img_to_array(img)/255.
#             img = ndimage.rotate(img, degree)
#             print(img)
            batch_imgs.append(img)
           # Store class
            label = load_img(self.label_dir + self.labels[i] ,target_size=self.dim)
            label = img_to_array(label)[:,:,0]
            label = label != 0
            label = mp.binary_erosion(mp.binary_erosion(label))
            label = mp.binary_dilation(mp.binary_dilation(mp.binary_dilation(label)))
            label = np.expand_dims((label)*1 , axis=2)
            batch_labels.append(label)
            
        return np.array(batch_imgs,dtype = np.float32 ) ,np.array(batch_labels , dtype = np.float32 )
    
train_generator = DataGenerator(train_imgs,train_img_dir,train_masks,train_mask_dir,batch_size=num_batch_size, dim=(224,224,3) ,shuffle=True)
train_steps = train_generator.__len__()

val_generator = DataGenerator(val_imgs,val_img_dir,val_masks,val_mask_dir,batch_size=num_batch_size, dim=(224,224,3) ,shuffle=True)
val_steps = val_generator.__len__()

### 搭建模型

In [11]:
def conv_block(tensor, nfilters, size=3, padding='same', initializer="he_normal"):
    x = Conv2D(filters=nfilters, kernel_size=(size, size), padding=padding, kernel_initializer=initializer)(tensor)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Conv2D(filters=nfilters, kernel_size=(size, size), padding=padding, kernel_initializer=initializer)(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    return x


def deconv_block(tensor, residual, nfilters, size=3, padding='same', strides=(2, 2)):
    y = Conv2DTranspose(nfilters, kernel_size=(size, size), strides=strides, padding=padding)(tensor)
    y = concatenate([y, residual], axis=3)
    y = conv_block(y, nfilters)
    return y


def Unet(h, w, filters):
# down
    input_layer = Input(shape=(h, w, 3), name='image_input')
    conv1 = conv_block(input_layer, nfilters=filters)
    conv1_out = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = conv_block(conv1_out, nfilters=filters*2)
    conv2_out = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = conv_block(conv2_out, nfilters=filters*4)
    conv3_out = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = conv_block(conv3_out, nfilters=filters*8)
    conv4_out = MaxPooling2D(pool_size=(2, 2))(conv4)
    conv4_out = Dropout(0.5)(conv4_out)
    conv5 = conv_block(conv4_out, nfilters=filters*16)
    conv5 = Dropout(0.5)(conv5)
# up
    deconv6 = deconv_block(conv5, residual=conv4, nfilters=filters*8)
    deconv6 = Dropout(0.5)(deconv6)
    deconv7 = deconv_block(deconv6, residual=conv3, nfilters=filters*4)
    deconv7 = Dropout(0.5)(deconv7) 
    deconv8 = deconv_block(deconv7, residual=conv2, nfilters=filters*2)
    deconv9 = deconv_block(deconv8, residual=conv1, nfilters=filters)
    output_layer = Conv2D(filters=1, kernel_size=(1, 1), activation='sigmoid')(deconv9)
    # using sigmoid activation for binary classification
    model = Model(inputs=input_layer, outputs=output_layer, name='Unet')
    return model

## **Nest-Unet** 

In [12]:
def conv_block(input_tensor, num_filters):
    x = Conv2D(num_filters, (3, 3), padding='same')(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(num_filters, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

def upsample_concat_block(input_tensor, skip_tensor, num_filters):
    x = UpSampling2D((2, 2))(input_tensor)
    x = concatenate([x, skip_tensor])
    x = conv_block(x, num_filters)
    return x

def NestedUNet(input_shape, num_filters=64, dropout_rate=0.5):
    inputs = Input(input_shape)

    # Encoder (downsampling)
    conv1 = conv_block(inputs, num_filters)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = conv_block(pool1, num_filters*2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = conv_block(pool2, num_filters*4)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = conv_block(pool3, num_filters*8)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
    pool4 = Dropout(dropout_rate)(pool4)

    # Bottleneck
    conv5 = conv_block(pool4, num_filters*16)
    conv5 = Dropout(dropout_rate)(conv5)

    # Decoder (upsampling)
    up6 = upsample_concat_block(conv5, conv4, num_filters*8)
    up6 = Dropout(dropout_rate)(up6)

    up7 = upsample_concat_block(up6, conv3, num_filters*4)
    up7 = Dropout(dropout_rate)(up7)

    up8 = upsample_concat_block(up7, conv2, num_filters*2)

    up9 = upsample_concat_block(up8, conv1, num_filters)

    # Output layer
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(up9)

    model = Model(inputs=inputs, outputs=outputs, name='NestedUNet')
    return model

### 实例化

In [5]:
model = Unet(224 , 224 , 64)
model.summary()

Model: "Unet"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_input (InputLayer)    [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d_19 (Conv2D)          (None, 224, 224, 64)         1792      ['image_input[0][0]']         
                                                                                                  
 batch_normalization_18 (Ba  (None, 224, 224, 64)         256       ['conv2d_19[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_18 (Activation)  (None, 224, 224, 64)         0         ['batch_normalization_18[0]

In [13]:
model = NestedUNet(input_shape=(256, 256, 3), num_filters=64)
model.summary()

Model: "NestedUNet"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 conv2d_38 (Conv2D)          (None, 256, 256, 64)         1792      ['input_2[0][0]']             
                                                                                                  
 batch_normalization_36 (Ba  (None, 256, 256, 64)         256       ['conv2d_38[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_36 (Activation)  (None, 256, 256, 64)         0         ['batch_normalization

### 设置评价指标

In [14]:
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def iou(y_true, y_pred, smooth=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
    union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
    return K.mean((intersection + smooth) / (union + smooth), axis=0)

def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

###  网络编译

In [17]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[dice_coef, iou, recall, precision])
callbacks = []

###  模型训练

In [ ]:
num_epoch = 40
results = model.fit(train_generator, steps_per_epoch=train_steps,epochs=num_epoch,callbacks=callbacks,validation_data=val_generator,validation_steps=val_steps)

/tmp/ipykernel_1112/887113016.py:58: DeprecationWarning: Please use `binary_erosion` from the `scipy.ndimage` namespace, the `scipy.ndimage.morphology` namespace is deprecated.
  label = mp.binary_erosion(mp.binary_erosion(label))
/tmp/ipykernel_1112/887113016.py:59: DeprecationWarning: Please use `binary_dilation` from the `scipy.ndimage` namespace, the `scipy.ndimage.morphology` namespace is deprecated.
  label = mp.binary_dilation(mp.binary_dilation(mp.binary_dilation(label)))


Epoch 1/40


### 模型保存

In [ ]:
history_data = results.history
df = pd.DataFrame(history_data)
df.to_csv('result/training_history.csv')
# 保存整个模型，包括结构、权重和训练配置
model.save('result/complete_model.h5')
# 保存模型的权重
model.save_weights('result/complete_model_weights_only.h5')